# pystac-client Tutorial



In [ ]:
import pystac
from pystac_client import Client

In [ ]:
# public STAC URLs

PC_URL = 'https://planetarycomputer.microsoft.com/api/stac/v1'
CMR_URL = 'https://cmr.earthdata.nasa.gov/stac'

# custom headers
headers = []

In [ ]:
cat = Client.open(CMR_URL, headers=headers)

In [ ]:
print(cat)

In [ ]:
for kitten in cat.get_children():
    print(kitten)

In [ ]:
cat = Client.open('https://cmr.earthdata.nasa.gov/stac/ASF')

In [ ]:
for col in cat.get_collections():
    print(col)